In [ ]:

!pip install pennylane

import pennylane as qml
from pennylane import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

df = pd.read_csv("https://github.com/RajKumarPandey/qml_notebooks/blob/main/synthetic_pki_dataset.csv")
X = df.drop("label", axis=1).values
y = df["label"].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = X_scaled[:, :2]
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit(x, weights):
    qml.RY(x[0], wires=0)
    qml.RY(x[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RY(weights[0], wires=0)
    qml.RY(weights[1], wires=1)
    return qml.expval(qml.PauliZ(1))

def cost(weights, features, labels):
    predictions = [circuit(x, weights) for x in features]
    return np.mean((predictions - labels) ** 2)

weights = np.array([0.01, 0.01], requires_grad=True)
opt = qml.GradientDescentOptimizer(0.2)
for _ in range(30):
    weights = opt.step(lambda w: cost(w, X_train, y_train), weights)

preds = [1 if circuit(x, weights) > 0 else 0 for x in X_test]
print("Accuracy:", accuracy_score(y_test, preds))
